In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

2024-07-03 10:33:03.836749: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-03 10:33:03.885755: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-03 10:33:04.701556: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Load and scale the dateset

In [2]:
data = pd.read_csv('/home/kxlu/Documents/xai_compare/data/time/ETTh1.csv').drop('date',axis=1)
scalers={}
for i in data.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(data[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    data[i]=s_s
train_df = data.iloc[:int(data.shape[0]*0.8)]
test_df = data.iloc[train_df.shape[0]:]

Transform the dataset into training samples using rolling window

In [3]:
def split_series(series, n_past, n_future):
  
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [4]:
lag =320; horizon = 32
X_train,y_train = split_series(train_df.values, n_past=lag, n_future=horizon)
X_test,y_test = split_series(test_df.values, n_past=lag, n_future=horizon)
n_features = X_train.shape[-1]

### LSTM

In [5]:
encoder_inputs = tf.keras.layers.Input(shape=(lag, n_features))
encoder = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

decoder_inputs = tf.keras.layers.RepeatVector(horizon)(encoder_outputs1[0])

decoder = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder)

model = tf.keras.models.Model(encoder_inputs,decoder_outputs)

model.summary()

2024-07-03 10:33:13.978610: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5800 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:1c:00.0, compute capability: 8.6
2024-07-03 10:33:13.979417: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 8792 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:1d:00.0, compute capability: 8.6
2024-07-03 10:33:13.980067: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 8023 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:3f:00.0, compute capability: 8.6


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 320, 7)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 100),     │     43,200 │ input_layer[0][0] │
│                     │ (None, 100),      │            │                   │
│                     │ (None, 100)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector       │ (None, 32, 100)   │          0 │ lstm[0][0]        │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 32, 100)   │     80,400 │ repeat_vector[0]… │
│                     │                   │            │ lstm[0][1],       │
│                     │                   │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 32, 7)     │        707 │ lstm_1[0][0]      │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 124,307 (485.57 KB)

 Trainable params: 124,307 (485.57 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history=model.fit(X_train,y_train,epochs=5,batch_size=32)

Epoch 1/5


2024-07-03 10:33:20.162473: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8905


425/425 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - loss: 0.0225
Epoch 2/5
425/425 ━━━━━━━━━━━━━━━━━━━━ 17s 40ms/step - loss: 0.0111
Epoch 3/5
425/425 ━━━━━━━━━━━━━━━━━━━━ 17s 40ms/step - loss: 0.0103
Epoch 4/5
425/425 ━━━━━━━━━━━━━━━━━━━━ 17s 40ms/step - loss: 0.0096
Epoch 5/5
425/425 ━━━━━━━━━━━━━━━━━━━━ 16s 37ms/step - loss: 0.0090


In [29]:
from omnixai.data.timeseries import Timeseries
def split_series_X(series, n_past):
    X = list()
    for window_start in range(len(series)):
        past_end = window_start + n_past
        if past_end >= len(series):
            break
        past = series[window_start:past_end, :]
        X.append(past)
    return np.stack(X)

def preprocess(ts):
    return ts[np.newaxis,:,:]

def predict_1(x):
    if isinstance(x,Timeseries):
        x = x.to_numpy()
    if isinstance(x,np.ndarray) and len(x.shape) == 2:
        x = x[np.newaxis,:,:]
    return model(x)[0]

InvalidIndexError: (None, slice(None, None, None), slice(None, None, None))

In [30]:
from omnixai.data.timeseries import Timeseries
from omnixai.explainers.timeseries import ShapTimeseries
# Initialize a TimeseriesExplainer
explainers = ShapTimeseries(
    training_data=Timeseries.from_pd(train_df),
    predict_function = predict_1,
    mode='forecasting',
)
# Generate local explanations
test_instance = Timeseries.from_pd(test_df[:320])
local_explanations = explainers.explain(test_instance)

ValueError: operands could not be broadcast together with shapes (22400,) (100,) 